In [ ]:
!pip install selenium
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
import csv
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    return driver

def process_url(url):
    driver = web_driver()

    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "pp-overview-item__title")))
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        company_name_element = soup.find('div', class_='pp-logo-company')
        if company_name_element:
            company_name = company_name_element.find('img')['alt']
        else:
            company_name = "Company name not found"

        year_founded_element = soup.find('li', text='Year Founded')
        year_founded = year_founded_element.find_next('span', class_='pp-overview-item__title').get_text(strip=True)

        employees_element = soup.find('li', text='Employees')
        employees = employees_element.find_next('span', class_='pp-overview-item__title').get_text(strip=True)

        description_element = soup.find('p', class_='pp-description_title')
        description = description_element.get_text(strip=True) if description_element else "Description not found"

        website_element = soup.find('h5', text='Website')
        website_url = website_element.find_next('a')['href'] if website_element else "Website URL not found"



        #ownership_status_element = soup.find('div', class_='font-weight-normal')
        #ownership_status = ownership_status_element.get_text(strip=True) if ownership_status_element else "Ownership status not found"
        # Find the <p> element containing the funding information
        # Extract Funding
        ul_tag = soup.find('ul', class_='list-type-none pp-faqs-table')

        if ul_tag:
    # Find all <p> tags within the <ul> tag
            p_tags = ul_tag.find_all('p', class_='XL-7 M-12 mb-xl-0')

            for p_tag in p_tags:
        # Extract the text content of each <p> tag
                text_content = p_tag.get_text(strip=True)

        # Check if the text content contains the funding information
                if "raised $" in text_content:
                    funding_info = text_content
                if "industry" in text_content:
                    industry_info = text_content

                if "competitor" in text_content:
                    competitors_info = text_content
                else:
                    competitors_info = "not found"




        data = {
            'URL': url,
            'Company Name': company_name,
            'Year Founded': year_founded,
            'Employees': employees,
            'Description': description,
            'Website': website_url,
            "funding_amount" :funding_info,
            "industry" : industry_info,
            "competitors" :competitors_info
            }

        return data

    except Exception as e:
        print(f"Error processing URL: {url}")
        print(f"Error message: {str(e)}")
        return None

    finally:
        driver.quit()
#add path here of csv file
csv_file_path = '/content/Lead data - pitchbook_output.csv'
data_list = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        url = row['link'].strip()
        if url.startswith('https://pitchbook.com'):
            print(f"Processing URL: {url}")
            data = process_url(url)
            if data:
                data_list.append(data)
            print("Completed processing for:", url)
            print("-" * 50)
            time.sleep(2)

df = pd.DataFrame(data_list)

output_csv_path = 'output_data.csv'
df.to_csv(output_csv_path, index=False)
print(f"Data saved to {output_csv_path}")

print(df)
